In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm_notebook
np.set_printoptions(suppress=True)
from sklearn import preprocessing
from tqdm import tqdm
import decimal
from itertools import combinations
from random import shuffle
from time import sleep
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import gc
import datetime
from sklearn.model_selection import train_test_split, KFold, GroupKFold
import os
from sklearn.metrics import roc_auc_score
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
import random
import lightgbm as lgb

In [2]:
#Always seed the randomness of this universe
def seed_everything(seed=51):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(seed=51)

In [4]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [5]:
%%time
train = pd.read_csv('../input/train4.csv')
print("train finished")

train finished
CPU times: user 44.5 s, sys: 3.27 s, total: 47.8 s
Wall time: 47.8 s


In [6]:
%%time
test = pd.read_csv('../input/test4.csv')
print("test finished")

test finished
CPU times: user 38.4 s, sys: 2.22 s, total: 40.6 s
Wall time: 40.6 s


In [7]:
#sample_submission
sample_submission = pd.read_csv('../input/sample_submission.csv')
print(train.shape,test.shape,sample_submission.shape)

(590540, 805) (506691, 804) (506691, 2)


In [8]:
#y = train['isFraud']
#del train['isFraud']
gc.collect()
print(train.shape,test.shape)

(590540, 804) (506691, 804)


In [ ]:
"""importances = pd.read_csv('./importances/importances1.csv')
print(importances.shape)
importances = importances[:700]
features = [x for x in list(importances['index']) if x not in ['TransactionID','TransactionDT']]
train = train[features]
test = test[features]
"""

In [9]:
gc.collect()

0

In [12]:
print(train.shape,test.shape)

(590540, 805) (506691, 804)


In [13]:
train.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D10,D11,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_17,id_18,id_19,id_20,id_30,id_31,id_32,id_33,id_35,id_36,id_37,id_38,DeviceInfo,dayofyear,hour,dayofweek,day,card1_999999,card1_fraudulent,card2_999999,card2_fraudulent,card3_999999,card3_185.0,card4_discover,card5_nan,card5_999999,card5_137.0,card5_118.0,card5_100.0,card5_150.0,card5_229.0,card5_117.0,card5_166.0,card5_126.0,is_debit,ProductCD_C,ProductCD_W,addr1_999999,addr1_nan,addr1_888888.0,addr1_191.0,addr1_184.0,addr1_436.0,addr1_143.0,addr1_205.0,addr1_194.0,addr1_494.0,addr1_444.0,addr2_999999,addr2_nan,addr2_96.0,addr2_999999.0,TransactionAmt_decimal,TransactionAmt_check,dist1_999999,dist1_nan,dist1_91.0,dist1_130.0,dist1_116.0,dist1_108.0,dist1_86.0,dist1_95.0,dist1_114.0,dist1_33.0,dist1_12.0,dist1_27.0,dist1_50.0,dist1_55.0,dist1_54.0,dist1_42.0,dist1_47.0,dist1_35.0,dist1_43.0,dist1_49.0,dist2_999999,dist2_nan,dist2_0.0,dist2_9.0,dist2_1.0,dist2_10.0,dist2_68.0,dist2_999999.0,dist2_7.0,P_emaildomain_prefix,P_emaildomain_suffix,P_emaildomain_lesscategory,P_emaildomain_999999,P_emaildomain_nan,P_emaildomain_mail.com,P_emaildomain_outlook.es,P_emaildomain_aim.com,P_emaildomain_outlook.com,P_emaildomain_roadrunner.com,P_emaildomain_verizon.net,P_emaildomain_att.net,P_emaildomain_sbcglobal.net,P_emaildomain_rocketmail.com,P_emaildomain_windstream.net,P_emaildomain_hotmail.fr,P_emaildomain_protonmail,R_emaildomain_prefix,R_emaildomain_suffix,R_emaildomain_lesscategory,R_emaildomain_999999,R_emaildomain_nan,R_emaildomain_protonmail,R_emaildomain_outlook.com,R_emaildomain_outlook.es,R_emaildomain_icloud.com,R_emaildomain_gmail.com,R_emaildomain_bellsouth.net,R_emaildomain_sbcglobal.net,R_emaildomain_msn.com,R_emaildomain_verizon.net,R_emaildomain_hotmail.fr,R_emaildomain_att.net,group_1_mean,group_1_median,group_1_min,group_1_max,group_1_sum,group_1_std,group_1_kurtosis,group_2_mean,group_2_median,group_2_min,group_2_max,group_2_sum,group_2_std,group_2_kurtosis,group_3_mean,group_3_median,group_3_min,group_3_max,group_3_sum,group_3_std,group_3_kurtosis,group_4_mean,group_4_median,group_4_min,group_4_max,group_4_sum,group_4_std,group_4_kurtosis,gro

In [14]:
cat_cols = ['ProductCD','card1','card2','card3','card4','card5','dist1','dist2','P_emaildomain','R_emaildomain','DeviceInfo']
print(len(cat_cols))

11


In [ ]:
for tuple_ in tqdm(list(combinations(cat_cols,2))):
    f1 = tuple_[0]
    f2 = tuple_[1]
    column_name = f1 + '_' + f2
    train[column_name] = train[f1].apply(str) + '_' + train[f2].apply(str)
    test[column_name] = test[f1].apply(str) +  '_' +test[f2].apply(str)
    only_train = len([ x for x in train[column_name].unique() if x not in test[column_name].unique() ])
    only_test = len([ x for x in test[column_name].unique() if x not in train[column_name].unique() ])
    shared = train[column_name].nunique() + test[column_name].nunique() - only_train - only_test
    print( column_name,shared,only_train,train[column_name].nunique(),round(shared/train[column_name].nunique(),3) )
    print( column_name,shared,only_test,test[column_name].nunique(),round(shared/test[column_name].nunique(),3) )
print(train.shape,test.shape)

In [20]:
"""
for tuple_ in tqdm(list(combinations(cat_cols,2))):
    f1 = tuple_[0]
    f2 = tuple_[1]
    column_name = f1 + '_' + f2
    train[column_name] = train[f1].apply(str) + '_' + train[f2].apply(str)
    test[column_name] = test[f1].apply(str) +  '_' +test[f2].apply(str)
    le = LabelEncoder()
    le.fit(list(train[column_name]) + list(test[column_name]))
    train[column_name] = le.transform(train[column_name])
    test[column_name] = le.transform(test[column_name])
print(train.shape,test.shape)
"""

100%|██████████| 55/55 [01:04<00:00,  1.18s/it]

(590540, 860) (506691, 859)


# Training

In [21]:
y = train['isFraud']
del train['isFraud']

In [25]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.02,
                    'num_leaves': 2**8,
                    'max_depth':12,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.5,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':2000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 51,
                    'early_stopping_rounds':100, 
                } 

In [28]:
N = 5
kf = KFold(n_splits=N)

importance = pd.DataFrame(np.zeros((train.shape[1], N)), columns=['Fold_{}'.format(i) for i in range(1, N + 1)], index=train.columns)
scores = []
y_pred = np.zeros(test.shape[0])
oof = np.zeros(train.shape[0])

for fold, (trn_idx, val_idx) in enumerate(kf.split(train, y), 1):
    print('Fold {}'.format(fold))
          
    trn_data = lgb.Dataset(train.iloc[trn_idx, :].values, label=y.iloc[trn_idx].values)
    val_data = lgb.Dataset(train.iloc[val_idx, :].values, label=y.iloc[val_idx].values)   
    
    clf = lgb.train(lgb_params, trn_data ,valid_sets=[trn_data, val_data], verbose_eval=100)

    predictions = clf.predict(train.iloc[val_idx, :].values) 
    importance.iloc[:, fold - 1] = clf.feature_importance()
    oof[val_idx] = predictions

    score = roc_auc_score(y.iloc[val_idx].values, predictions)
    scores.append(score)
    print('Fold {} ROC AUC Score {}\n'.format(fold, score))

    y_pred += clf.predict(test) / N
    
    del trn_data, val_data, predictions
    gc.collect()
    
print('Average ROC AUC Score {} [STD:{}]'.format(np.mean(scores), np.std(scores)))

Fold 1


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.956324	valid_1's auc: 0.892265
[200]	training's auc: 0.979909	valid_1's auc: 0.907358
[300]	training's auc: 0.988062	valid_1's auc: 0.916027
[400]	training's auc: 0.992098	valid_1's auc: 0.920168
[500]	training's auc: 0.994837	valid_1's auc: 0.922587
[600]	training's auc: 0.996554	valid_1's auc: 0.923491
[700]	training's auc: 0.997665	valid_1's auc: 0.924373
[800]	training's auc: 0.998408	valid_1's auc: 0.925455
[900]	training's auc: 0.998982	valid_1's auc: 0.926126
[1000]	training's auc: 0.999384	valid_1's auc: 0.926292
[1100]	training's auc: 0.999611	valid_1's auc: 0.926092
Early stopping, best iteration is:
[1031]	training's auc: 0.99946	valid_1's auc: 0.926597
Fold 1 ROC AUC Score 0.9265974578747068

Fold 2


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.953969	valid_1's auc: 0.911219
[200]	training's auc: 0.979668	valid_1's auc: 0.926194
[300]	training's auc: 0.988938	valid_1's auc: 0.932097
[400]	training's auc: 0.99289	valid_1's auc: 0.93458
[500]	training's auc: 0.995508	valid_1's auc: 0.93671
[600]	training's auc: 0.996929	valid_1's auc: 0.937782
[700]	training's auc: 0.998015	valid_1's auc: 0.938478
[800]	training's auc: 0.998766	valid_1's auc: 0.938931
[900]	training's auc: 0.999218	valid_1's auc: 0.939557
[1000]	training's auc: 0.999543	valid_1's auc: 0.939927
[1100]	training's auc: 0.999747	valid_1's auc: 0.940341
[1200]	training's auc: 0.999856	valid_1's auc: 0.940344
Early stopping, best iteration is:
[1110]	training's auc: 0.999767	valid_1's auc: 0.940475
Fold 2 ROC AUC Score 0.9404752976443973

Fold 3


...

Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.957174	valid_1's auc: 0.912361
[200]	training's auc: 0.981208	valid_1's auc: 0.92589
[300]	training's auc: 0.98984	valid_1's auc: 0.930251
[400]	training's auc: 0.993615	valid_1's auc: 0.932065
[500]	training's auc: 0.995713	valid_1's auc: 0.932689
[600]	training's auc: 0.997102	valid_1's auc: 0.932805
[700]	training's auc: 0.998122	valid_1's auc: 0.933536
[800]	training's auc: 0.99888	valid_1's auc: 0.933729
[900]	training's auc: 0.999312	valid_1's auc: 0.933721
Early stopping, best iteration is:
[837]	training's auc: 0.999091	valid_1's auc: 0.933834
Fold 3 ROC AUC Score 0.933833924875208

Fold 4


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.952887	valid_1's auc: 0.925081
[200]	training's auc: 0.979484	valid_1's auc: 0.941154
[300]	training's auc: 0.988562	valid_1's auc: 0.946829
[400]	training's auc: 0.993036	valid_1's auc: 0.949815
[500]	training's auc: 0.995771	valid_1's auc: 0.951238
[600]	training's auc: 0.997337	valid_1's auc: 0.951927
[700]	training's auc: 0.998344	valid_1's auc: 0.952281
Early stopping, best iteration is:
[695]	training's auc: 0.998305	valid_1's auc: 0.952298
Fold 4 ROC AUC Score 0.9522983521364924

Fold 5


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:116: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:121: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.954626	valid_1's auc: 0.90169
[200]	training's auc: 0.980298	valid_1's auc: 0.919483
[300]	training's auc: 0.989129	valid_1's auc: 0.925075
[400]	training's auc: 0.993365	valid_1's auc: 0.927515
[500]	training's auc: 0.995731	valid_1's auc: 0.929492
[600]	training's auc: 0.997205	valid_1's auc: 0.929828
[700]	training's auc: 0.998158	valid_1's auc: 0.929971
[800]	training's auc: 0.998936	valid_1's auc: 0.930174
[900]	training's auc: 0.99937	valid_1's auc: 0.930336
Early stopping, best iteration is:
[803]	training's auc: 0.998948	valid_1's auc: 0.930356
Fold 5 ROC AUC Score 0.9303559181951323

Average ROC AUC Score 0.9367121901451873 [STD:0.009034199872049967]


In [29]:
print(roc_auc_score(y,oof))

0.9359838164608781


In [30]:
importance['avg'] = importance.mean(axis=1)

In [31]:
importance = importance.sort_values(by='avg',ascending=False)

In [32]:
importance = importance.reset_index(drop=False)

In [33]:
importance.head()

,index,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,avg
0,TransactionID,7131,7608,5846,4642,5409,6127.2
1,TransactionAmt,6815,7459,5370,4360,5363,5873.4
2,card1,5370,5706,4318,3432,3872,4539.6
3,addr1,4837,5553,3774,3224,3726,4222.8
4,day,4183,4515,2969,2500,3063,3446.0


In [35]:
importance.to_csv('../importances/importances5.csv',index=False)

In [36]:
sample_submission.head()

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5


In [37]:
sample_submission['isFraud'] = y_pred

In [38]:
sample_submission.head()

,TransactionID,isFraud
0,3663549,0.000818
1,3663550,0.000959
2,3663551,0.001961
3,3663552,0.001280
4,3663553,0.001131


In [40]:
sample_submission.to_csv('../predictions/pred5.csv',index=False)

In [42]:
!kaggle competitions submit -c ieee-fraud-detection -f ../predictions/pred5.csv -m "pred5"

100%|███████████████████████████████████████| 14.1M/14.1M [00:33<00:00, 441kB/s]
Successfully submitted to IEEE-CIS Fraud Detection